# Compare Running vs. Finetuning

In [ ]:
# Install the requirements in Google Colab
# !pip install transformers datasets trl huggingface_hub

from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import setup_chat_format
import torch
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
# Authenticate to Hugging Face
# from huggingface_hub import login

# login("")

/home/ai-makina/.pyenv/versions/smol-course-2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 135M Model

### Running 135M Model

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

prompt = "What is the meaning of life?"

model_name = "HuggingFaceTB/SmolLM2-135M"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

outputs = model.generate(**inputs, max_new_tokens=200)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


What is the meaning of life?

The meaning of life is to live a life that is full of joy, happiness, and fulfillment. It is to live a life that is full of love, compassion, and kindness. It is to live a life that is full of purpose, meaning, and purpose. It is to live a life that is full of purpose, meaning, and purpose.

What is the meaning of life?

The meaning of life is to live a life that is full of purpose, meaning, and purpose. It is to live a life that is full of joy, happiness, and fulfillment. It is to live a life that is full of purpose, meaning, and purpose. It is to live a life that is full of purpose, meaning, and purpose.

What is the meaning of life?

The meaning of life is to live a life that is full of purpose, meaning, and purpose. It is to live a life that is full of joy, happiness,


### Finetuning 135M Model

In [3]:
# Load the model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)

# Set up the chat format
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2-135M-Arch-Talk-Chat-Model"
finetune_tags = ["arch-talk", "smoltalk", "everyday-conversations"]

# Split Dataset
# Load the dataset first - using 'everyday-conversations' config for chat training
ds = load_dataset("HuggingFaceTB/smoltalk", "everyday-conversations", split="train")
# Split the dataset for training and evaluation
ds = ds.train_test_split(test_size=0.5, seed=42)

print(f"Training samples: {len(ds['train'])}")
print(f"Test samples: {len(ds['test'])}")
print(f"Example conversation: {ds['train'][0]['messages']}")

# Configure the SFTTrainer
sft_config = SFTConfig(
    output_dir="./sft_output",
    max_steps=1000,  # Adjust based on dataset size and desired training duration
    per_device_train_batch_size=8,  # Set according to your GPU memory capacity
    learning_rate=5e-5,  # Common starting point for fine-tuning
    logging_steps=10,  # Frequency of logging training metrics
    save_steps=100,  # Frequency of saving model checkpoints
    eval_steps=50,  # Frequency of evaluation
    use_mps_device=(
        True if device == "mps" else False
    ),  # Use MPS for mixed precision training on Apple Silicon
    hub_model_id=finetune_name,  # Set a unique name for your model
    push_to_hub=False,  # Automatically push the model to Hugging Face Hub
    hub_private_repo=False,  # Make the model public
    report_to=None,  # Disable wandb/tensorboard logging for simplicity
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=ds["train"],
    eval_dataset=ds["test"],
    processing_class=tokenizer,  # Use processing_class instead of tokenizer
)

Training samples: 1130
Test samples: 1130
Example conversation: [{'content': 'Hi there', 'role': 'user'}, {'content': 'Hello! How can I help you today?', 'role': 'assistant'}, {'content': "I'm having trouble with my work schedule. I have a lot of night shifts this week.", 'role': 'user'}, {'content': 'Night shifts can be challenging. Are you finding it hard to adjust to the new sleep schedule?', 'role': 'assistant'}, {'content': 'Yes, I am. Do you have any tips for staying awake during my shift?', 'role': 'user'}, {'content': 'Try to get some fresh air during your breaks, and avoid heavy meals before your shift starts. Also, stay hydrated by drinking plenty of water throughout the night.', 'role': 'assistant'}, {'content': 'That sounds helpful, thanks.', 'role': 'user'}]


In [8]:
# Train the model
trainer.train()

# Save the model
trainer.save_model(f"./{finetune_name}")

Step,Training Loss
10,2.163400
20,1.445800
30,1.267600
40,1.238500
50,1.136600
60,1.126600
70,1.119500
80,1.116300
90,1.078600
100,1.072400


model.safetensors: 100%|██████████| 538M/538M [01:17<00:00, 6.97MB/s] 


In [ ]:
# trainer.push_to_hub(tags=finetune_tags)

CommitInfo(commit_url='https://huggingface.co/salhernandez/SmolLM2-135M-Arch-Talk-Chat-Model/commit/91d2481d83617c875982b6d763cdcad012509a8c', commit_message='End of training', commit_description='', oid='91d2481d83617c875982b6d763cdcad012509a8c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/salhernandez/SmolLM2-135M-Arch-Talk-Chat-Model', endpoint='https://huggingface.co', repo_type='model', repo_id='salhernandez/SmolLM2-135M-Arch-Talk-Chat-Model'), pr_revision=None, pr_num=None)